In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from konlpy.tag import Twitter

import pandas as pd
import pickle

##### Load Dataset
- 0:정치, 1:경제, 2:사회, 3:생활/문화, 4:세계, 5:IT/과학

In [2]:
article_df = pd.read_pickle("article_2016-06-01.plk")
print(len(article_df))
article_df.tail(3)

2327


,newsid,oid,newspaper,title,link,comment,likeit,content,date,category
2324,3179706,23,조선일보,"[어수웅의 르네상스人] 종이 사전을 삼킨 남자, &#39;웹 사전&#39;을 낳다",http://news.naver.com/main/read.nhn?mode=LSD&m...,0,2,"[정철 카카오 웹 사전 기획자] 책 '검색, 사전을 삼키다' 펴내… 연세대에서 사전...",2016-06-01,5
2325,2976797,20,동아일보,"中, M&A로 특허 포식… 한국 미래산업 삼킨다",http://news.naver.com/main/read.nhn?mode=LSD&m...,13,9,"[동아일보]中, 로봇-IoT-바이오 등 M&A; 규모… 올해 5월에 이미 작년 기록...",2016-06-01,5
2326,2620983,25,중앙일보,구글·애플 대항마…토종 앱 장터 원스토어 떴다,http://news.naver.com/main/read.nhn?mode=LSD&m...,123,9,"이통 3사, 네이버 손잡고 시장 40% 점유 목표| 양강 구도에 도전장…앱 생태계 ...",2016-06-01,5


##### Split Train, Test

In [3]:
X_train, X_test, y_train, y_test = train_test_split(article_df.content, article_df.category,\
                                                    test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(2094, 233, 2094, 233)

In [4]:
pos_tagger = Twitter()
def tokenize_pos(doc):
    return ["/".join(t) for t in pos_tagger.pos(doc)]

In [5]:
clf = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize_pos)),
    ('clf',MultinomialNB(alpha=0.01)),
])

In [6]:
model = clf.fit(X_train, y_train)

##### Confusion Matrix, Classfication Report

In [7]:
from sklearn.metrics import confusion_matrix, classification_report

In [8]:
y_pred = model.predict(X_test)

In [9]:
confusion_matrix(y_test, y_pred)

array([[40,  0,  3,  1,  2,  0],
       [ 1, 36,  0,  3,  0,  1],
       [ 1,  4, 74,  4,  0,  0],
       [ 0,  0,  1, 20,  0,  0],
       [ 1,  1,  0,  0, 27,  0],
       [ 0,  1,  1,  0,  0, 11]])

In [10]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.93      0.87      0.90        46
          1       0.86      0.88      0.87        41
          2       0.94      0.89      0.91        83
          3       0.71      0.95      0.82        21
          4       0.93      0.93      0.93        29
          5       0.92      0.85      0.88        13

avg / total       0.90      0.89      0.89       233



##### Prediction

In [ ]:
X_test = X_test.reset_index(drop=True)
X_test[0][:50], X_test[1][:50], X_test[2][:50]

In [ ]:
result = model.predict([X_test[0], X_test[1], X_test[2]])
result

In [ ]:
classification_dict = {
    0:"정치", 
    1:"경제", 
    2:"사회", 
    3:"생활/문화", 
    4:"세계", 
    5:"IT/과학",
}

for idx, category in enumerate(result):
    print(classification_dict[category], "-", X_test[idx][:50])

##### Save and Load Model 

In [ ]:
pickle.dump(model, open("classification.plk", "wb"))

In [ ]:
load_model = pickle.load(open("classification.plk", "rb"))

In [ ]:
str1 = "네이버와 카카오는 드론 기술 발전에 주력"
str2 = "요즘 환율과 주가는 예측이 불가"
load_model.predict([str1, str2])

In [ ]:
classification_dict[load_model.predict([str1])[0]],\
classification_dict[load_model.predict([str2])[0]]

In [ ]:
load_model.predict_proba([str2])